<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js014_crystalMMD2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#### simulator( html + css + js ) + control( python )

In [1]:
#@title js014_crystalMMD2D / def exec_html_js() ... exec me first

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js014_crystalMMD2D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js014_crystalMMD2D
//    Copyright(C) 2017-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2017.05.07 created, last updated on 2018.11.14
//    ver 0.0.1  2019.01.13 v1, last updated on 2021.05.02
//    ver 0.0.2  2021.10.29 v2, last updated on 2021.10.29
//    ver 0.0.3  2023.03.04 v3, last updated on 2023.08.16
//
// --------------------  molecular dynamics 2D
//
//  method: velocity Verlet Algorithm
//    (1) vi = vi + (Fi/mi)*(0.5dt)
//    (2) ri = ri + vi*dt
//    (3) calculation Fi <- {r1,r2,...,rn} Fi=sum(Fij,j=1 to n),Fij=F(ri-rj)
//    (4) vi = vi + (Fi/mi)*(0.5dt)
//    goto (1)
//
//  potential: Morse V(r) = D*((1-EXP(-A*(r-r0)))^2-1)
//                        = D*(EXP(-2*A*(r-r0))-2*EXP(-A*(r-r0)))
//    (D:dissociation energy, r0:bond length, A:width parameter { A=SQR(k/(2*D)) }
//             force F(r) = -dV(r)/dr
//                        = 2*D*A*y*(y-1), y=EXP(-A*(r-r0))
//
// --------------------
*/

const crystalMMD2D = (function(){ // ====================  crystalMMD2D Module  ====================

	const g_AMU = 1.66053904e-27;		// (kg) atomic mass unit
	const g_kB = 1.380649e-23;			// (J/K) Boltzmann's constant
	const g_EE = 1.602176634e-19;		// (C) electron charge, energy : 1(eV) = EE(J)
	const g_nMax = 2000;				// array max

	let g_nParticles = 100;				// number of particles
	let g_sysTime = 0.0;				// (s) system time
	let g_timeStep =  5.0*1.0e-15;		// (s) time step
	let g_xMax = 6.0E-9;				// (m) x-Box size
	let g_yMax = 6.0E-9;				// (m) y-Box size
	let g_kineticEnergy = 0.0;			// (J) total kinetic energy
	let g_potentialEnergy = 0.0;		// (J) total potential energy

	const g_xx = dim1( g_nMax );		// (m) x-component of i-th particle position
	const g_yy = dim1( g_nMax );		// (m) y-component of i-th particle position
	const g_vx = dim1( g_nMax );		// (m/s) x-component of i-th particle velocity
	const g_vy = dim1( g_nMax );		// (m/s) y-component of i-th particle velocity
	const g_ffx = dim1( g_nMax );		// (N) x-component of total force applied i-th particle
	const g_ffy = dim1( g_nMax );		// (N) y-component of total force applied i-th particle

	function dim1( n ) {
		return new Float64Array( n );
	}

	//--- material data

	let g_mass = 55.847*g_AMU;			// (kg) mass of Fe
	let g_ddMorse = 0.4174*g_EE;		// (J) D of Morse potential : energy of dissociation
	let g_aaMorse = 1.3885e10;			// (1/m) A of Morse potential : width parameter
	let g_r0Morse = 2.845e-10;			// (m) r0 of Morse potential : bond length


	// --------------------  set initial condition  --------------------

	function setInitialCondition( boxSizeInNM, xtalSizeInNM, contTemp ) {
		g_sysTime = 0.0;
		g_xMax = boxSizeInNM*1.0e-9;
		g_yMax = boxSizeInNM*1.0e-9;
		const s = 0.5*(boxSizeInNM-xtalSizeInNM)*1.0e-9;
		g_nParticles = setCrystalBlock(0, s, s, xtalSizeInNM*1.0e-9, xtalSizeInNM*1.0e-9, Math.PI/4.0);
		ajustVelocity(contTemp);
	}

	function setCrystalBlock(ii, x0, y0, xLen, yLen, theta) {
		let iip = ii;
		const a = 0.98*g_r0Morse;
		const b = 0.866025*a;
		let leng = xLen;
		if (leng<yLen) leng = yLen;
		leng = 1.5*leng;
		const nx = Math.floor(leng/b) + 1;
		const ny = Math.floor(leng/a) + 1;
		const sth = Math.sin(theta);
		const cth = Math.cos(theta);
		for (let i=0; i<nx; i++) {
			const x = b*i - leng/2.0;
			for (let j=0; j<ny; j++) {
				let y = a*j - leng/2.0;
				if ((i%2)==1) y = y + 0.5*a;
				const xp = x0 + xLen/2.0 + cth*x - sth*y;
				const yp = y0 + yLen/2.0 + sth*x + cth*y;
				if (xp>=x0 && xp<=x0+xLen && yp>=y0 && yp<=y0+yLen) {
					setParticle(iip, xp, yp);
					iip = iip + 1;
				}
			}
		}
		return iip;
	}

	function setParticle(i, x, y) {
		g_xx[i] = x;
		g_yy[i] = y;
		g_vx[i] = 200.0*normalRandom3();
		g_vy[i] = 200.0*normalRandom3();
		g_ffx[i] = 0.0;
		g_ffy[i] = 0.0;
	}

	// normal distributed random number: -3.0 <= normalRandom3() < 3.0
	function normalRandom3() {
		return (Math.random()+Math.random()+Math.random()+Math.random()+Math.random()+Math.random()-3.0);
	}


	// --------------------  time evolution  --------------------

	function timeEvolution( tempMode, contTemp ) {
		if (tempMode==1) ajustVelocity(contTemp);
		for (let i=0; i<20; i++) {
			g_sysTime += g_timeStep;
			moveParticles(g_timeStep);
		}
	}

	function moveParticles(dt) {
		const nn=g_nParticles;
		const a = 0.5*dt/g_mass;
		for (let i=0; i<nn; i++) {
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
			g_xx[i] += g_vx[i]*dt;
			g_yy[i] += g_vy[i]*dt;
		}
		calcForce();
		for (let i=0; i<nn; i++) {
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
		}
		g_kineticEnergy = 0.0;
		for (let i=0; i<nn; i++) {
			g_kineticEnergy += 0.5*g_mass*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]);
		}
	}

	function calcForce() { // calc. force and potential energy
		const nn=g_nParticles, s05 = 0.5*3.418e-10; // Ar sigma=3.418e-10
		g_potentialEnergy = 0.0;
		for (let i=0; i<nn; i++) {
			g_ffx[i]=0;g_ffy[i]=0;
		}
		for (let i=0; i<nn-1; i++) {
			for (let j=i+1; j<nn; j++) {
				const xij=g_xx[i]-g_xx[j], yij=g_yy[i]-g_yy[j];
				const r = Math.sqrt(xij*xij+yij*yij);
				// calc. force and potential energy
				const y = Math.exp(-g_aaMorse*(r-g_r0Morse)); // y = EXP(-A*(r-r0))
				g_potentialEnergy += g_ddMorse*y*(y-2.0); // V(r) = D*((1-y)^2-1) = D*y*(y-2)
				const f = 2.0*g_ddMorse*g_aaMorse*y*(y-1); // F(r) = 2*D*A*y*(y-1)
				//
				const fxij = f*xij/r;
				const fyij = f*yij/r;
				g_ffx[i] += fxij;
				g_ffy[i] += fyij;
				g_ffx[j] -= fxij;
				g_ffy[j] -= fyij;
			}
		}
		for (let i=0; i<nn; i++) {
			g_ffx[i] += boundaryForce(g_xx[i]+s05)+boundaryForce(g_xx[i]-g_xMax-s05);
			g_ffy[i] += boundaryForce(g_yy[i]+s05)+boundaryForce(g_yy[i]-g_yMax-s05);
		}
	}

	function boundaryForce(r) { // boundary:L-J type; epsilon = 0.5*epsilonOfAr, sigma = sigmaOfAr
		const ri = (3.418e-10/r); // sigmaOfAr = 3.418e-10
		const r6 =ri*ri*ri*ri*ri*ri;
		g_potentialEnergy += 4.0*0.5*1.711e-21*r6*(r6-1.0); // epsilonOfAr = 1.711e-21
		return (24.0*0.5*1.711e-21*r6*(2.0*r6-1.0)/r);
	}


	// --------------------  utility  --------------------

	function systemTemperature() {
		const nn = g_nParticles;
		let ek = 0.0;  // kinetic energy (J)
		for (let i=0; i<nn; i++) {
			ek = ek + 0.5*g_mass*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]);
		}
		return ek/(nn*g_kB);
	}

	function ajustVelocity(temp) {
		const nn=g_nParticles, a = Math.sqrt(temp/systemTemperature());
		for (let i=0; i<nn; i++) {
			g_vx[i] = a*g_vx[i];
			g_vy[i] = a*g_vy[i];
		}
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( boxSizeInNM, xtalSizeInNM, contTemp )
		evolve:			timeEvolution,			// timeEvolution( tempMode, contTemp )

		setTemp:		ajustVelocity,			// ajustVelocity( temp )

		getSysParam:	function() { return [ g_nParticles, g_timeStep, g_xMax, g_yMax ]; },
		getNow:			function() { return [ g_sysTime, systemTemperature(), g_kineticEnergy, g_potentialEnergy ]; },
		getAtomData:	function() { return [ 3, 0.5*g_r0Morse/1.12246, g_r0Morse ]; }, // [kind, rCollision, bondLength ]
		getPosition:	function(i) { return [ g_xx[i], g_yy[i] ]; },
		getVelocity:	function(i) { return [ g_vx[i], g_vy[i] ]; },
		getForce:		function(i) { return [ g_ffx[i], g_ffy[i] ]; },
	};

})(); // ====================  crystalMMD2D end  ====================


const js014 = (function(){ // ====================  js Module  ====================

	const theModule = crystalMMD2D;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;
	let ctx;

	let v_theme = 3; //  = kind  - Fe
	let v_BoxSizeInNM = 4.0;
	let v_xtalSizeInNM = 2.5;
	let v_contTemp = 300.0;
	let v_tempMode = 0;			// 0: adiabatic, 1: temp.control

	let p_nParticles, p_timeStep, p_xMax, p_yMax; // = theModule.getSysParam();
	let sysTime, temperature, kineticEnergy, potentialEnergy;

	let nCalc = 1;
	let dispMode = 1;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;
  let breakFlag = false;


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_BoxSizeInNM, v_xtalSizeInNM, v_contTemp );
			[ p_nParticles, p_timeStep, p_xMax, p_yMax ] = theModule.getSysParam();
		}

		if ( !pauseFlag ) {
			for(let i=0; i<nCalc; i++ ) {
				theModule.evolve( v_tempMode, v_contTemp );
			}
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( v_tempMode, v_contTemp );
		}

		draw( ctx, dispMode );

		requestAnimationFrame(animate);
	}


	// --------------------  draw  --------------------

	const atomColor = { Fe:"#00c0ff" };

	function draw( ctx, dispMode ) {
		const xp = 40, yp = 5, xSize = 400, ySize = 400, scale = xSize/p_xMax, yTextPos= 430;

		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		ctx.strokeStyle = "#888800";
		ctx.strokeRect( xp, yp, xSize, ySize );

		if ( dispMode==0 ) {
			drawBalls( ctx, scale, xp, yp );
		} else if ( dispMode==1 || dispMode==2 ) {
			strokeBalls( ctx, scale, xp, yp );
			drawBonds( ctx, dispMode+1, scale, xp, yp );
		}

		// caption
		[ sysTime, temperature, kineticEnergy, potentialEnergy ] = theModule.getNow();
		ctx.fillStyle = "#888888";
		ctx.fillText(`time = ${(sysTime*1.0e12).toFixed(1)} (ps)`, xp, yTextPos);
		ctx.fillText(`Temp = ${temperature.toFixed(1)} (K)`, xp+200, yTextPos);
		ctx.fillText(`atom: Fe,   N = ${p_nParticles}`, xp, yTextPos+20);
		ctx.fillText(`cont. Temp = ${v_contTemp.toFixed(0)} (K)`, xp+200, yTextPos+20);
		ctx.fillText(`Box = ${(p_xMax*1.0e9).toFixed(1)} x ${(p_yMax*1.0e9).toFixed(1)} (nm)`, xp, yTextPos+40);
		ctx.fillText(`Energy = ${(kineticEnergy+potentialEnergy).toExponential(4)} (J)`, xp+200, yTextPos+40);
		//document.getElementById("text_caption").innerHTML = "minimum molecular dynamics code"

	}

	function drawBalls( ctx, scale, xp, yp ) {
		const nn = p_nParticles, yMax = p_yMax, twoPi = 2.0*Math.PI;
		const rCollision = theModule.getAtomData()[1];
		for (let i=0; i<nn; i++) {
			let x, y; [ x, y ] = theModule.getPosition(i);
			ctx.fillStyle = atomColor.Fe;
			ctx.beginPath();
			ctx.arc(x*scale+xp, (yMax-y)*scale+yp, rCollision*scale, 0, twoPi, false);
			ctx.fill();
		}
	}

	function strokeBalls( ctx, scale, xp, yp ) {
		const nn = p_nParticles, yMax = p_yMax, twoPi = 2.0*Math.PI;
		const d0 = theModule.getAtomData()[2]; // bond length
		const rBond = 0.5*d0;
		for (let i=0; i<nn; i++) {
			let x, y; [ x, y ] = theModule.getPosition(i);
			ctx.strokeStyle = "#333333";
			ctx.beginPath();
			ctx.arc(x*scale+xp, (yMax-y)*scale+yp, rBond*scale, 0, twoPi, false);
			ctx.stroke();
		}
	}

	function drawBonds( ctx, bondMode, scale, xp, yp ) {
		const nn = p_nParticles, yMax = p_yMax, pi = Math.PI;
		const d0 = theModule.getAtomData()[2]; // bond length
		const dc = d0*1.2;

		for (let i=0; i<nn-1; i++) {
			let xi, yi; [ xi, yi ] = theModule.getPosition(i);
			for (let j=i+1; j<nn; j++) {
				let xj, yj; [ xj, yj ] = theModule.getPosition(j);
				const xij = xi-xj, yij = yi-yj;
				const dij = Math.sqrt(xij*xij+yij*yij);
				if ( dij<dc ) {
					let hue;
					if ( (bondMode & 1)==0 ) { // bond color -- length
						hue = Math.floor((dij/d0-1.0)*900+120);
					} else if ( (bondMode & 1)==1 ) { // bond color -- direction
						const th = 3.0*(Math.atan2(yij,xij)+0.5*pi)/pi;
						hue = Math.floor((th-Math.floor(th))*360.0);
					}
					if (hue<0) hue = 0;
					if (hue>270) hue = 270;
					const color = `hsl(${hue},100%,50%)`;
					drawLine(ctx, xi*scale+xp, (yMax-yi)*scale+yp, xj*scale+xp, (yMax-yj)*scale+yp, color );
				}
			}
		}
	}

	function drawLine( ctx, x1, y1, x2, y2, color ) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	}


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {
		v_theme = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}


	function setTempMode() {
		v_tempMode = 0 + document.getElementById("slct_tempMode").selectedIndex;
	}

	function setContTemp() {  // range mixing
		v_contTemp = Number(document.getElementById("range_temp").value);
		document.getElementById("text_temp").innerHTML = " " + v_contTemp.toFixed(0);
	}

	function setDispMode() {
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
	}

	function setNcalc() {
		nCalc = 1 + document.getElementById("slct_nCalc").selectedIndex;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_theme = theme
    document.getElementById("slct_theme").selectedIndex = theme;
    resetFlag = true;
  }

  function pysetTempMode( mode ) {
    v_tempMode = mode;
    document.getElementById("slct_tempMode").selectedIndex = mode;
  }

  function pysetTemperature(temp) {
    theModule.setTemp(temp);
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, temperature, kineticEnergy, potentialEnergy ];
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		// dom control
		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		setTheme:		setTheme,		// setTheme()
		setTempMode:	setTempMode,	// setTempMode()
		setContTemp:	setContTemp,	// setContTemp()
		setDispMode:	setDispMode,	// setDispMode()
		setNcalc:		setNcalc,		// setNcalc()

	  breakLoop: breakLoop, // breakLoop();
	  pysetTheme: pysetTheme, // pysetTheme( theme )
	  pysetTempMode: pysetTempMode, // pysetTempMode( mode )
	  pysetTemperature: pysetTemperature, // pysetTemperature( temp )
	  pysetDispMode: pysetDispMode, // pysetDispMode( mode )
	  pygetData: pygetData, // pygetData( pyMsg ) : return [ sysTime, temperature, kineticEnergy, potentialEnergy ]
	};

})(); // ====================  js014 module end  ====================


const js = js014;
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js014] crystal (Morse potential) MD2D</p>
<canvas ID="canvas_box" style="background-color: #000000;" width="480" height="480"></canvas>
<br>

<label>theme: </label> Fe
    <span style="margin-right: 220px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>Temp mode:</label>
<select id="slct_tempMode" onChange="js.setTempMode()">
<option selected>adiabatic</option><option>Temp.control</option>
</select>
<br>

<label>cont.Temp:</label><label id="text_temp"> 300</label>
<input type="range" id="range_temp" min="10" max="600" value="300" step="2"
style="width:360px" oninput="js.setContTemp()">
<br>

<label> disp. mode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option>ball</option><option selected>bond length</option><option>bond direction</option>
</select>
    <span style="margin-right: 80px;"></span>
<label>speed(nCalc/frame):</label>
<select id="slct_nCalc" onChange="js.setNcalc()">
<option selected>1</option><option>2</option><option>3</option><option>4</option>
<option>5</option><option>6</option>
</select>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt


# exec html-js code
exec_html_js()
print("-- start --")
time.sleep(3)

# python control
for i in range(10):
  [ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = kineticEnergy + potentialEnergy
  print(
    "i = {},\t".format(i),
    "time = {:.2f} (ps), ".format(sysTime*1e12),
    "temp = {:.2f} (K), ".format(sysTemp),
    "energy = {:.5g} (J)".format(energy) )
  time.sleep(2)
#

time.sleep(1)
# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# exec html-js code, and python control / change dispMode

import time
import numpy as np
import matplotlib.pyplot as plt


# exec html-js code
exec_html_js()
print("-- start --")
time.sleep(3)

# change theme and dispMode
dispModeList = [ 'ball', 'bond length', 'bond direction' ]
print( "-- theme: Fe --" )
for dispMode in [ 0, 1, 2 ]:
  eval_js( 'js.pysetDispMode({})'.format(dispMode) )
  print( "-- -- dispMode:", dispModeList[dispMode], "--" )
  time.sleep(3)
  # get data and display
  for i in [ 0, 1, 2, 3 ]:
    [ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
    energy = kineticEnergy + potentialEnergy
    print(
      "i = {},\t".format(i),
      "time = {:.2f} (ps), ".format(sysTime*1e12),
      "temp = {:.2f} (K), ".format(sysTemp),
      "energy = {:.5g} (J)".format(energy) )
    time.sleep(2)


time.sleep(1)
# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# exec html-js code, and python control / system heat up and plot

import time
import numpy as np
import matplotlib.pyplot as plt


# exec html-js code
exec_html_js()
print("-- start --")
time.sleep(1)

# chage dispMode
dispMode = 0 # ball
eval_js( 'js.pysetDispMode({})'.format(dispMode) )
print("-- dispMode==0 ball --")
time.sleep(3)

# chage dispMode
dispMode = 1 # bond-length
eval_js( 'js.pysetDispMode({})'.format(dispMode) )
print("-- dispMode==1 bond-length --")
time.sleep(3)


timeList = []
tempList = []
energyList = []

for i in range(20):
  # set temperature
  temp = 300 + 100*i
  eval_js( 'js.pysetTemperature({})'.format(temp) )
  time.sleep(3)

  # get data and display
  [ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = kineticEnergy + potentialEnergy

  timeList.append(sysTime*1e12)
  tempList.append(sysTemp)
  energyList.append(energy*1e20)

  if i%2==0:
    print(
      "i = {},\t".format(i),
      "time = {:.2f} (ps), ".format(sysTime*1e12),
      "temp = {:.2f} (K), ".format(sysTemp),
      "energy = {:.5g} (J)".format(energy) )
#
time.sleep(3)


# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

# plot
plt.plot( timeList, tempList )
plt.xlabel('time (ps)')
plt.ylabel('temp (K)')
plt.show()